In [1]:
# Import 
import pandas as pd 
import numpy as np
import mne
import matplotlib.pyplot as plt 

In [8]:
# Import dataset - DREAM 
df = pd.read_csv("../dataset/DREAM/Datasets.csv", index_col=0)
rec_df = pd.read_csv("../dataset/DREAM/Data records.csv", index_col=0)
people_df = pd.read_csv("../dataset/DREAM/People.csv")

file = "../dataset/DREAM/Zhang & Wamsley 2019/Data/PSG/subject026_Morning.edf"
data = mne.io.read_raw_edf(file)
dream_df = data.get_data()

# Metadata 
info = data.info
channels = data.ch_names
raw_df = pd.DataFrame(dream_df)
raw_df.index = channels

Extracting EDF parameters from c:\Users\PC\Documents\Personal Projects\AI_Honor_track\project\TUe-Honors-MedAI\dataset\DREAM\Zhang & Wamsley 2019\Data\PSG\subject026_Morning.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [11]:
# Record info
rec_df \
    .loc[rec_df.Filename == "subject026_Morning.edf"] \
    .iloc[0]

Set ID                                         1
Amendment                                      0
Filename                  subject026_Morning.edf
Case ID                               26_Morning
Subject ID                                    26
Experience                                   2.0
Treatment group                              NaN
Duration                                    73.0
EEG sample rate                              400
Number of EEG channels                        58
Last sleep stage                             2.0
Has EOG                                        1
Has EMG                                        1
Has ECG                                        0
Proportion artifacts                         NaN
Time of awakening                       08:52:24
Subject age                                 18.0
Subject sex                                    0
Subject healthy                                1
Has more data                                  1
Remarks             

#### 1. Multitaper spectral analysis 

Decompose the raw EEG data in DREAM dataset into spectrogram similar to the PhysioNet dataset SDB. 
1. Divide into 2s epoch - Be aware of the sampling rate
2. Appy multitaper spectral analysis to drive spectrogram of 0.5 Hz band between 0 Hz and 100 Hz for each window of 2s (epoch)

In [29]:
# Filter 
eeg_df = raw_df.loc["Fp1-REF"].copy().values
sample_df = eeg_df[:800] # Get first epoch of 2s 

In [ ]:
# 